In [ ]:
## import numpy as np # linear algebra
from matplotlib import pyplot as plt
#We need these in this file:
import sys
sys.path.append('cropping')
sys.path.append('scripts')
import load_read_name_extractor as lrne
import SVM_classifier_general as svm_general

#Taget fra længere nede i koden:
import torch
from torchvision import transforms
from PIL import Image
import random
import itertools
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import DeepFunctions as df
from random import shuffle
from torch.utils.data import Dataset, DataLoader, random_split
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from torchvision import transforms
from IPython.display import display
import os
import json
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Call the function to list GPU names
df.list_gpu_names()
#R
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
trainDataloader = torch.load('data/train_loader.pth')
valDataloader = torch.load('data/val_loader.pth')
testDataloader = torch.load('data/test_loader.pth')
lenthdataset = torch.load('data/lengthsdataset.pth')

In [ ]:
bestnet = NeuralNetwork()
path_model = "models/best_model_state_Acc_0.704_epoch_133.0.pt"
path_result = "results/0.0008_0_5_0.1_0.3_71.65%_results.json"
bestnet.load_state_dict(torch.load(path_model))
bestnet.to(device)

In [ ]:
cm1, acc1 = df.create_confusion_matrix(trainDataloader, bestnet, device=device)
title1 = 'Confusion matrix for training set'

cm2, acc2 = df.create_confusion_matrix(valDataloader, bestnet, device=device)
title2 = 'Confusion matrix for validation set'

cm3, acc3 = df.create_confusion_matrix(testDataloader, bestnet, device=device)
title3 = 'Confusion matrix for test set'

df.plot_3_confusion_matrices(cm1, cm2, cm3, acc1, acc2, acc3, num_classes, title1, title2, title3)

In [ ]:
df.plot_loss_and_accuracy(df.transfer_to_cpu(trainLossList), df.transfer_to_cpu(trainAccList), df.transfer_to_cpu(valLossList), df.transfer_to_cpu(valAccList))